# **Optimization Methods for Clustering**
### **Members:** Luca Tusini (2092227), Davide Christian Mancosu Bustos (2089208), Karim Eugenio Hamdar (2092041)



---
## **Loading Data**

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
def read_clq_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
        edges = []

    for line in lines:
        line = line.strip()

        # Skip comments and empty lines
        if line.startswith('c') or not line:
            continue

        # Process problem line
        if line.startswith('p'):
            parts = line.split()
            if len(parts) == 4 and (parts[1] == 'edge' or parts[1] == 'col'):
                num_vertices = int(parts[2])
                num_edges = int(parts[3])
                print(f"Number of vertices: {num_vertices}")
                print(f"Number of edges: {num_edges}")

        # Process edge lines
        if line.startswith('e'):
            parts = line.split()
            if len(parts) == 3:
                vertex1 = int(parts[1])
                vertex2 = int(parts[2])
                edges.append((vertex1, vertex2))

    adjacency_matrix = np.zeros((num_vertices, num_vertices), dtype=int)
    for vertex1, vertex2 in edges:
        adjacency_matrix[vertex1-1][vertex2-1] = 1
        adjacency_matrix[vertex2-1][vertex1-1] = 1

    return num_vertices, num_edges, adjacency_matrix

In [3]:
file_name = 'hamming8-4.clq'
num_vertices, num_edges, adjacency_matrix = read_clq_file(file_name)

Number of vertices: 256
Number of edges: 20864


---
## **L2 Regularized Max-Clique Problem**

In [4]:
def objective_function(x, A):
    return np.dot(x.T, np.dot(A, x)) + 0.5 * np.dot(x.T, x)

def gradient(x, A):
    return -2 * np.dot(A, x) - x

def LMO(grad, n):
    ik = np.argmin(grad)
    y = np.zeros(n)
    y[ik] = 1
    return y

# Compute the L constant as the max eigenvalue of the matrix: 2 * A + I
def compute_L(A):
    I = np.eye(A.shape[0])
    M = 2 * A + I
    eigenvalues = np.linalg.eigvals(M)
    L = np.max(np.abs(eigenvalues))
    return L

---
### 1) Classic Frank-Wolfe Algorithm

In [5]:
def FW(A, max_iter=1000, line_search='diminishing stepsize'):

    n = A.shape[0]
    results = []

    if line_search == 'lipschitz constant dependent stepsize':
        L = compute_L(A)

    for initial_node in range(n):

        x = np.zeros(n)
        x[initial_node] = 1  # Initial guess

        for iter in range(max_iter):

            grad = gradient(x, A)

            y = LMO(grad, n)

            d_k = y - x

            if line_search == 'diminishing stepsize':
                alpha = 2 / (2 + iter)
            elif line_search == 'lipschitz constant dependent stepsize':
                alpha = min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1)

            x_new = x + (alpha * d_k)
            x = x_new

        clique_vertices = np.where(x > 0)[0]
        results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [6]:
starting_time = time.time()
r_FW = FW(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_FW = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_FW}s')
print(f'Max clique found: {r_FW[1]}, for initial node: {r_FW[0]}')

Execution time: 57.03s
Max clique found: 16, for initial node: 0


---
### 2) Away-step Frank-Wolfe Algorithm

In [7]:
def away_step_FW(A, max_iter=1000, line_search='diminishing stepsize'):

    n = A.shape[0]
    results = []

    if line_search == 'lipschitz constant dependent stepsize':
        L = compute_L(A)

    for initial_node in range(n):

      x = np.zeros(n)

      x[initial_node] = 1
      S = {initial_node}  # Initial active set


      for k in range(max_iter):

          grad = gradient(x,A)

          # Frank-Wolfe direction
          x_FW = LMO(grad, n)
          d_FW = x_FW - x

          # Away-step direction
          i_AS = max(S, key=lambda i: grad[i])
          x_AS = np.zeros(n)
          x_AS[i_AS] = 1
          d_AS = x - x_AS

          g_FW = grad @ d_FW
          g_AS = grad @ d_AS

          if g_FW <= g_AS:
              d_k = d_FW
              alpha_max = 1
          else:
              d_k = d_AS
              alpha_max = x[i_AS] / (1 - x[i_AS])

          if line_search == 'diminishing stepsize':
              alpha_k = min(alpha_max, 2 / (2 + k))
          elif line_search == 'lipschitz constant dependent stepsize':
              alpha_k = min(alpha_max, min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1))

          x = x + alpha_k * d_k

          S = {i for i in range(n) if x[i] > 0} # Update active set

      clique_vertices = np.where(x > 0)[0]
      results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [8]:
starting_time = time.time()
r_AS = away_step_FW(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_AS = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_AS}s')
print(f'Max clique found: {r_AS[1]}, for initial node: {r_AS[0]}')

Execution time: 75.03s
Max clique found: 16, for initial node: 0


---
### 3) Pair Wise Frank-Wolfe Algorithm

In [9]:
def pairwise_FW(A, max_iter=1000, line_search='diminishing stepsize', starting_node=0):

    n = A.shape[0]
    results = []

    if line_search == 'lipschitz constant dependent stepsize':
        L = compute_L(A)

    for initial_node in range(n):

        x = np.zeros(n)
        x[starting_node] = 1

        S = {starting_node}  # Initial active set

        for k in range(max_iter):

            grad = gradient(x, A)

            # Frank-Wolfe direction
            x_FW = LMO(grad, n)
            d_FW = x_FW - x

            # Away-step direction
            i_AS = max(S, key=lambda i: grad[i])
            x_AS = np.zeros(n)
            x_AS[i_AS] = 1
            d_AS = x - x_AS

            # Pairwise FW direction
            d_k = d_FW + d_AS

            alpha_max = np.inf
            for i in range(n):
                if d_k[i] > 0:
                    alpha_max = min(alpha_max, (1 - x[i]) / d_k[i])
                elif d_k[i] < 0:
                    alpha_max = min(alpha_max, -x[i] / d_k[i])

            if line_search == 'diminishing stepsize':
                alpha_k = min(alpha_max, 2 / (2 + k))
            elif line_search == 'lipschitz constant dependent stepsize':
                alpha_k = min(alpha_max, min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1))

            x = x + alpha_k * d_k

            S = {i for i in range(n) if x[i] > 0} # Update active set

        clique_vertices = np.where(x > 0)[0]
        results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [10]:
starting_time = time.time()
r_PW = pairwise_FW(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_PW = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_PW}s')
print(f'Max clique found: {r_PW[1]}, for initial node: {r_PW[0]}')

Execution time: 135.36s
Max clique found: 16, for initial node: 0


---
### 4) Model Comparison

In [11]:
print(f'--- 1) FW ({time_FW}s)')
print(f'Max clique found: {r_FW[1]}, for initial node: {r_FW[0]}\n------------------------------------------')
print(f'--- 2) Away Step FW ({time_AS}s)')
print(f'Max clique found: {r_AS[1]}, for initial node: {r_AS[0]}\n------------------------------------------')
print(f'--- 3) Pairwise FW ({time_PW}s)')
print(f'Max clique found: {r_PW[1]}, for initial node: {r_PW[0]}\n------------------------------------------')

--- 1) FW (57.03s)
Max clique found: 16, for initial node: 0
------------------------------------------
--- 2) Away Step FW (75.03s)
Max clique found: 16, for initial node: 0
------------------------------------------
--- 3) Pairwise FW (135.36s)
Max clique found: 16, for initial node: 0
------------------------------------------


---
## **L0 Regularized Max-Clique Problem**

In [12]:
def objective_function_2(x, A, alpha2, beta):
    return np.dot(x.T, np.dot(A, x)) + (alpha2 * np.sum(np.exp(-beta * x) - 1))

def gradient_2(x, A, alpha2, beta):
    return -2 * np.dot(A, x) + alpha2 * beta * np.exp(-beta * x)

def LMO(grad, n):
    ik = np.argmin(grad)
    y = np.zeros(n)
    y[ik] = 1
    return y

# Compute the L constant as: 2∥A∥ + α2*β
def compute_L_2(A, alpha2, beta):
    A_norm = np.linalg.norm(A, ord=2)  # 2-norm of A
    L = 2 * A_norm + alpha2 * beta
    return L

---
### 1) Classic Frank-Wolfe Algorithm

In [13]:
def FW_2(A, max_iter=1000, line_search='diminishing stepsize', alpha2=0.05, beta=0.5):

    n = A.shape[0]
    results = []

    for initial_node in range(n):

        x = np.zeros(n)
        x[initial_node] = 1  # Initial guess

        if line_search == 'lipschitz constant dependent stepsize':
          L = compute_L_2(A, alpha2, beta)

        for iter in range(max_iter):

            grad = gradient_2(x, A, alpha2, beta)

            y = LMO(grad, n)

            d_k = y - x

            if line_search == 'diminishing stepsize':
                alpha = 2 / (2 + iter)
            elif line_search == 'lipschitz constant dependent stepsize':
                alpha = min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1)

            x_new = x + (alpha * d_k)
            x = x_new

        clique_vertices = np.where(x > 0)[0]
        results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [14]:
starting_time = time.time()
r_FW_2 = FW_2(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_FW_2 = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_FW_2}s')
print(f'Max clique found: {r_FW_2[1]}, for initial node: {r_FW_2[0]}')

Execution time: 40.66s
Max clique found: 16, for initial node: 0


---
### 2) Away-step Frank-Wolfe Algorithm

In [15]:
def away_step_FW_2(A, max_iter=1000, line_search='diminishing stepsize', alpha2=0.05, beta=0.5):

    n = A.shape[0]
    results = []

    if line_search == 'lipschitz constant dependent stepsize':
        L = compute_L(A, alpha2, beta)

    for initial_node in range(n):

      x = np.zeros(n)

      x[initial_node] = 1
      S = {initial_node}  # Initial active set


      for k in range(max_iter):

          grad = gradient_2(x, A, alpha2, beta)

          # Frank-Wolfe direction
          x_FW = LMO(grad, n)
          d_FW = x_FW - x

          # Away-step direction
          i_AS = max(S, key=lambda i: grad[i])
          x_AS = np.zeros(n)
          x_AS[i_AS] = 1
          d_AS = x - x_AS

          g_FW = grad @ d_FW
          g_AS = grad @ d_AS

          if g_FW <= g_AS:
              d_k = d_FW
              alpha_max = 1
          else:
              d_k = d_AS
              alpha_max = x[i_AS] / (1 - x[i_AS])

          if line_search == 'diminishing stepsize':
              alpha_k = min(alpha_max, 2 / (2 + k))
          elif line_search == 'lipschitz constant dependent stepsize':
              alpha_k = min(alpha_max, min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1))

          x = x + alpha_k * d_k

          S = {i for i in range(n) if x[i] > 0} # Update active set

      clique_vertices = np.where(x > 0)[0]
      results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [16]:
starting_time = time.time()
r_AS_2 = away_step_FW_2(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_AS_2 = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_AS_2}s')
print(f'Max clique found: {r_AS_2[1]}, for initial node: {r_AS_2[0]}')

Execution time: 80.31s
Max clique found: 16, for initial node: 0


---
### 3) Pair Wise Frank-Wolfe Algorithm

In [17]:
def pairwise_FW_2(A, max_iter=1000, line_search='diminishing stepsize', starting_node=0, alpha2=0.05, beta=0.5):

    n = A.shape[0]
    results = []

    if line_search == 'lipschitz constant dependent stepsize':
        L = compute_L(A, alpha2, beta)

    for initial_node in range(n):

        x = np.zeros(n)
        x[starting_node] = 1

        S = {starting_node}  # Initial active set

        for k in range(max_iter):

            grad = gradient_2(x, A, alpha2, beta)

            # Frank-Wolfe direction
            x_FW = LMO(grad, n)
            d_FW = x_FW - x

            # Away-step direction
            i_AS = max(S, key=lambda i: grad[i])
            x_AS = np.zeros(n)
            x_AS[i_AS] = 1
            d_AS = x - x_AS

            # Pairwise FW direction
            d_k = d_FW + d_AS

            alpha_max = np.inf
            for i in range(n):
                if d_k[i] > 0:
                    alpha_max = min(alpha_max, (1 - x[i]) / d_k[i])
                elif d_k[i] < 0:
                    alpha_max = min(alpha_max, -x[i] / d_k[i])

            if line_search == 'diminishing stepsize':
                alpha_k = min(alpha_max, 2 / (2 + k))
            elif line_search == 'lipschitz constant dependent stepsize':
                alpha_k = min(alpha_max, min(-np.dot(grad, d_k) / (L * np.dot(d_k, d_k)), 1))

            x = x + alpha_k * d_k

            S = {i for i in range(n) if x[i] > 0} # Update active set

        clique_vertices = np.where(x > 0)[0]
        results.append(len(clique_vertices))

    return (np.argmax(results), max(results))

In [18]:
starting_time = time.time()
r_PW_2 = pairwise_FW_2(adjacency_matrix, line_search='diminishing stepsize')
ending_time = time.time()

time_PW_2 = np.round(ending_time-starting_time,2)

print(f'Execution time: {time_PW_2}s')
print(f'Max clique found: {r_PW_2[1]}, for initial node: {r_PW_2[0]}')

Execution time: 143.8s
Max clique found: 16, for initial node: 0


---
### 4) Model Comparison

In [19]:
print(f'--- 1) FW ({time_FW_2}s)')
print(f'Max clique found: {r_FW_2[1]}, for initial node: {r_FW_2[0]}\n------------------------------------------')
print(f'--- 2) Away Step FW ({time_AS_2}s)')
print(f'Max clique found: {r_AS_2[1]}, for initial node: {r_AS_2[0]}\n------------------------------------------')
print(f'--- 3) Pairwise FW ({time_PW_2}s)')
print(f'Max clique found: {r_PW_2[1]}, for initial node: {r_PW_2[0]}\n------------------------------------------')

--- 1) FW (40.66s)
Max clique found: 16, for initial node: 0
------------------------------------------
--- 2) Away Step FW (80.31s)
Max clique found: 16, for initial node: 0
------------------------------------------
--- 3) Pairwise FW (143.8s)
Max clique found: 16, for initial node: 0
------------------------------------------
